In [24]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from math import ceil

## Experiment Design

In [4]:
#Storing baseline data
data = {"Metric Name": ["Cookies", "Clicks", "User-ids", "Click-through-probability", "Gross conversion", "Retention", "Net conversion"], 
     "Estimator": [40000, 3200, 660, 0.08, 0.20625, 0.53, 0.109313],
     "dmin": [3000, 240, -50, 0.01, -0.01, 0.01, 0.0075]}
df = pd.DataFrame(data)
df.set_index("Metric Name", inplace=True)

# Invariant and Evaluation metrics lists
invariant_metrics = ["Cookies", "Clicks", "Click-through-probability"]
evaluation_metrics = ["Gross conversion", "Retention", "Net conversion"]

In [5]:
# scale based on 5000 cookies
df.at["Cookies", "Scaled_Estimator"] = 5000
scale_factor = df.at['Cookies', 'Scaled_Estimator'] / df.at['Cookies', 'Estimator']
for i in ['Clicks', 'User-ids']:
    df.at[i,'Scaled_Estimator'] = scale_factor * df.at[i,'Estimator'] 

In [6]:
df

,Estimator,dmin,Scaled_Estimator
Metric Name,,,
Cookies,40000.000000,3000.0000,5000.0
Clicks,3200.000000,240.0000,400.0
User-ids,660.000000,-50.0000,82.5
Click-through-probability,0.080000,0.0100,NaN
Gross conversion,0.206250,-0.0100,NaN
Retention,0.530000,0.0100,NaN
Net conversion,0.109313,0.0075,NaN


In [7]:
# Standard Error
def get_se(n, p):
    '''Returns standard error for binomial distribution of given probobality p and sample size n'''
    return np.sqrt((p * (1-p)) / n)

In [16]:
for i, j in zip(['Clicks', 'User-ids', 'Clicks'], evaluation_metrics):
    df.at[j, "SE"] = get_se(df.at[i, 'Scaled_Estimator'], df.at[j, 'Estimator'])

In [17]:
df

,Estimator,dmin,Scaled_Estimator,SE,Sample_Size
Metric Name,,,,,
Cookies,40000.000000,3000.0000,5000.0,NaN,NaN
Clicks,3200.000000,240.0000,400.0,NaN,NaN
User-ids,660.000000,-50.0000,82.5,NaN,NaN
Click-through-probability,0.080000,0.0100,NaN,NaN,NaN
Gross conversion,0.206250,-0.0100,NaN,0.020231,50458.0
Retention,0.530000,0.0100,NaN,0.054949,78096.0
Net conversion,0.109313,0.0075,NaN,0.015602,55962.0


In [32]:
def calculate_sample_size(alpha, beta, p, dmin):
    '''Return sample size given alpha, beta, p and dmin'''
    z_alpha_over_2 = norm.ppf(1 - alpha/2)
    z_beta = norm.ppf(1 - beta)
    p1 = p
    p2 = p + dmin

    sample_size = ((z_alpha_over_2 + z_beta) ** 2) * (p1 * (1 - p1) + p2 * (1 - p2)) / ((p1 - p2) ** 2)
    return int(sample_size)

alpha = 0.05
beta = 0.2

for i, j in zip(evaluation_metrics, ['Clicks', 'User-ids', 'Clicks']):
    df.at[i, "Sample_Size"] = ceil(2 * calculate_sample_size(alpha, beta, df.at[i, 'Estimator'], df.at[i, 'dmin']) \
                                    / df.at[j, 'Estimator'] * df.at['Cookies', 'Estimator'])

In [37]:
df['estimated_exp_days'] = df['Sample_Size'] / df.at['Cookies', 'Estimator']

In [38]:
df

,Estimator,dmin,Scaled_Estimator,SE,Sample_Size,estimated_exp_days
Metric Name,,,,,,
Cookies,40000.000000,3000.0000,5000.0,NaN,NaN,NaN
Clicks,3200.000000,240.0000,400.0,NaN,NaN,NaN
User-ids,660.000000,-50.0000,82.5,NaN,NaN,NaN
Click-through-probability,0.080000,0.0100,NaN,NaN,NaN,NaN
Gross conversion,0.206250,-0.0100,NaN,0.020231,630725.0,15.768125
Retention,0.530000,0.0100,NaN,0.054949,4733091.0,118.327275
Net conversion,0.109313,0.0075,NaN,0.015602,699525.0,17.488125


## Experiment Analysis

In [42]:
control = pd.read_csv('data/Final Project Results - Control.csv')
exp = pd.read_csv('data/Final Project Results - Experiment.csv')

In [45]:
control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [46]:
exp.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [47]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB


In [48]:
exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB


### Sanity Check

In [63]:
sanity_check = pd.DataFrame(columns=['Control', 'Experiment','CI_lower','CI_upper','obs'],
                            index=['Pageviews','Clicks','CTP'])

In [74]:
# basic parameters
p = 0.5
alpha = 0.05
d_expect = 0

# cookies(pageviews) and clicks
for i in ['Pageviews', 'Clicks']:
    sanity_check.at[i, 'Control'] = control[i].sum()
    sanity_check.at[i, 'Experiment'] = exp[i].sum()
    se = np.sqrt((p * (1-p)) / (sanity_check.at[i, 'Control'] + sanity_check.at[i, 'Experiment']))
    sanity_check.at[i, 'CI_lower'] = p - norm.ppf(1-alpha/2) * se
    sanity_check.at[i, 'CI_upper'] = p + norm.ppf(1-alpha/2) * se
    sanity_check.at[i, 'obs'] = sanity_check.at[i, 'Control'] / (sanity_check.at[i, 'Control'] + sanity_check.at[i, 'Experiment'])

# CTP (click-through-probablity)
n_pool = 0
success_pool = 0
for i in ['Control', 'Experiment']:
    sanity_check.at['CTP', i] = sanity_check.at['Clicks', i] / sanity_check.at['Pageviews', i]
    n_pool += sanity_check.at['Pageviews', i]
    success_pool += sanity_check.at['Clicks', i]

p_pool = success_pool / n_pool
se_pool = np.sqrt(p_pool * (1-p_pool) * (1 / sanity_check.at['Pageviews', 'Control'] + 1 / sanity_check.at['Pageviews', 'Experiment']))
sanity_check.at['CTP', 'CI_lower'] = d_expect - norm.ppf(1-alpha/2) * se_pool
sanity_check.at['CTP', 'CI_upper'] = d_expect + norm.ppf(1-alpha/2) * se_pool
sanity_check.at['CTP', 'obs'] = sanity_check.at['CTP', 'Experiment'] - sanity_check.at['CTP', 'Control']

# sanity_check
sanity_check['pass_sanity_check'] = (sanity_check.obs <= sanity_check.CI_upper) & (sanity_check.obs >= sanity_check.CI_lower)


In [75]:
sanity_check

,Control,Experiment,CI_lower,CI_upper,obs,pass_sanity_check
Pageviews,345543,344660,0.49882,0.50118,0.50064,True
Clicks,28378,28325,0.495885,0.504115,0.500467,True
CTP,0.082126,0.082182,-0.001296,0.001296,0.000057,True


### Result Analysis

In [76]:
df

,Estimator,dmin,Scaled_Estimator,SE,Sample_Size,estimated_exp_days
Metric Name,,,,,,
Cookies,40000.000000,3000.0000,5000.0,NaN,NaN,NaN
Clicks,3200.000000,240.0000,400.0,NaN,NaN,NaN
User-ids,660.000000,-50.0000,82.5,NaN,NaN,NaN
Click-through-probability,0.080000,0.0100,NaN,NaN,NaN,NaN
Gross conversion,0.206250,-0.0100,NaN,0.020231,630725.0,15.768125
Retention,0.530000,0.0100,NaN,0.054949,4733091.0,118.327275
Net conversion,0.109313,0.0075,NaN,0.015602,699525.0,17.488125


In [91]:
# true sample size
exp.dropna().Pageviews.sum() + control.dropna().Pageviews.sum()

423525

In [108]:
control_clean = control.dropna()
exp_clean  = exp.dropna()

In [118]:
def get_CI_proportion(x_cont, x_exp, n_cont, n_exp, alpha=0.05, d_expect=0):
    """
    Calculate the confidence interval for the difference between two proportions.

    Args:
    - x_cont: Number of successes in the control group.
    - x_exp: Number of successes in the experimental group.
    - n_cont: Total observations in the control group.
    - n_exp: Total observations in the experimental group.
    - alpha: Significance level (default is 0.05).
    - expected_difference: Expected difference between proportions (default is 0).

    Returns:
    - Tuple containing the proportion in the control group, proportion in the experimental group,
      lower bound of the confidence interval, upper bound of the confidence interval, and the observed difference.
    """
    p_pool = (x_cont + x_exp) / (n_cont + n_exp)
    se_pool = np.sqrt(p_pool * (1-p_pool) * (1 / n_cont + 1/ n_exp))
    lower = d_expect - se_pool * norm.ppf(1 - alpha/2)
    upper = d_expect + se_pool * norm.ppf(1 - alpha/2)
    p_exp = x_exp / n_exp
    p_cont = x_cont / n_cont
    d_hat = p_exp - p_cont
    return p_cont, p_exp, lower, upper, d_hat

In [119]:
for key, v in {'Gross conversion' : ['Enrollments', 'Clicks'],
            'Net conversion' : ['Payments', 'Clicks']}.items():
    df.at[key, 'CI_lower'], df.at[key, 'CI_upper'], df.at[key, 'Observed'] = get_CI(control_clean[v[0]].sum(), exp_clean[v[0]].sum(),
                                                                                    control_clean[v[1]].sum(), exp_clean[v[1]].sum())

In [117]:
df

,Estimator,dmin,Scaled_Estimator,SE,Sample_Size,estimated_exp_days,CI_lower,CI_upper,Observed
Metric Name,,,,,,,,,
Cookies,40000.000000,3000.0000,5000.0,NaN,NaN,NaN,NaN,NaN,NaN
Clicks,3200.000000,240.0000,400.0,NaN,NaN,NaN,NaN,NaN,NaN
User-ids,660.000000,-50.0000,82.5,NaN,NaN,NaN,NaN,NaN,NaN
Click-through-probability,0.080000,0.0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross conversion,0.206250,-0.0100,NaN,0.020231,630725.0,15.768125,-0.008568,0.008568,-0.020555
Retention,0.530000,0.0100,NaN,0.054949,4733091.0,118.327275,NaN,NaN,NaN
Net conversion,0.109313,0.0075,NaN,0.015602,699525.0,17.488125,-0.006731,0.006731,-0.004874


In [114]:
exp_clean.Enrollments.sum() / exp_clean.Clicks.sum() - control_clean.Enrollments.sum() / control_clean.Clicks.sum()

-0.020554874580361565

0.19831981460023174